In [21]:
from cv2 import aruco
import cv2
import numpy as np
import matplotlib.pyplot as plt
import msgpack as mp
import msgpack_numpy as mpn
import os

from tqdm import tqdm
import csv

from joblib import Parallel, delayed

In [22]:
blur_parameters = [4, 6, 8, 10]
angle_parameters = [0, 30, 60, 90, 120, 150, 180]

In [23]:
num_processes = 20  # You can adjust this based on your requirements

In [36]:
raw_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_data_pth = os.path.join(raw_data_pth, 'dataset',"multi_class", "raw_data")

_raw_saved_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
_raw_saved_data_pth = os.path.join(_raw_saved_data_pth, 'dataset',"multi_class", "raw_data")
_raw_saved_data_pth = os.path.join(_raw_saved_data_pth, "images")
_raw_saved_data_list = os.listdir(_raw_saved_data_pth)


In [35]:
_blur_save_pth = os.path.dirname(os.path.dirname(_raw_saved_data_pth))
_blur_save_pth = os.path.join(_blur_save_pth, 'motion_blur')
_blur_save_img_pth = os.path.join(_blur_save_pth, "images")
_blur_save_label_pth = os.path.join(_blur_save_pth, "label")

if not os.path.exists(_blur_save_img_pth):
    os.makedirs(_blur_save_img_pth)

if not os.path.exists(_blur_save_label_pth):
    os.makedirs(_blur_save_label_pth)

In [26]:
def apply_motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    k = cv2.warpAffine(k, cv2.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0), (size, size) )  
    k = k * ( 1.0 / np.sum(k) )        
    return cv2.filter2D(image, -1, k)

In [27]:
def adding_blur_to_image(img_name):
    _image_path = os.path.join(_raw_saved_data_pth, img_name)
    image = cv2.imread(_image_path)
    #adding noise to twenty percent of the images
    _csv_rows = []
    
    if np.random.random() < 0.4:
        for _b in blur_parameters:
            for _a in angle_parameters:
                
                _blur_size = _b
                _blur_angle = _a
                
                image = apply_motion_blur(image, _blur_size, _blur_angle)        
                
                _image_path = os.path.join(_blur_save_img_pth, f"blur_s{_blur_size}_a{_blur_angle}_{img_name}")
                cv2.imwrite(_image_path, image)
                
                label_name = img_name.split(".")[0]
                label_path = os.path.join(_raw_saved_data_pth, "..","labels", f"{label_name}.txt")
                label_file = open(label_path, "r", newline='')
                label_reader = csv.reader(label_file, delimiter=" ")
                for _row in label_reader:
                    _csv_rows.append(_row)
                label_file.close()

                label_path = os.path.join(os.path.join(_blur_save_label_pth, "..", "labels"), f"blur_s{_blur_size}_a{_blur_angle}_{img_name.split('.')[0]}.txt")
                label_file = open(label_path, "w", newline='')
                label_writer = csv.writer(label_file, delimiter=" ")
                for _r in _csv_rows:
                    label_writer = csv.writer(label_file, delimiter=" ")
                    label_writer.writerow(_r)
                _csv_rows.clear()
                label_file.close()

    return 0
        
results = Parallel(n_jobs=num_processes, verbose=1)(delayed(adding_blur_to_image)(element) for element in _raw_saved_data_list)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s


[Parallel(n_jobs=20)]: Done 219 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 470 tasks      | elapsed:   11.9s
[Parallel(n_jobs=20)]: Done 820 tasks      | elapsed:   19.5s
[Parallel(n_jobs=20)]: Done 1270 tasks      | elapsed:   30.3s
[Parallel(n_jobs=20)]: Done 1820 tasks      | elapsed:   43.9s
[Parallel(n_jobs=20)]: Done 2470 tasks      | elapsed:   59.5s
[Parallel(n_jobs=20)]: Done 3220 tasks      | elapsed:  1.6min
[Parallel(n_jobs=20)]: Done 4070 tasks      | elapsed:  1.9min
[Parallel(n_jobs=20)]: Done 5020 tasks      | elapsed:  2.2min
[Parallel(n_jobs=20)]: Done 6070 tasks      | elapsed:  2.6min
[Parallel(n_jobs=20)]: Done 7220 tasks      | elapsed:  3.5min
[Parallel(n_jobs=20)]: Done 7985 out of 7985 | elapsed:  3.8min finished


In [37]:
# splitting dataset into train and validation and test
data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_pth = os.path.join(data_pth, 'dataset',"multi_class", "dataset_processed")

images_pth = os.path.join(data_pth, "images")
labels_pth = os.path.join(data_pth, "labels")

if not os.path.exists(images_pth):
    os.makedirs(os.path.join(images_pth, "train"))
    os.makedirs(os.path.join(images_pth, "val"))
    os.makedirs(os.path.join(images_pth, "test"))

if not os.path.exists(labels_pth):
    os.makedirs(os.path.join(labels_pth, "train"))
    os.makedirs(os.path.join(labels_pth, "val"))
    os.makedirs(os.path.join(labels_pth, "test"))
    
    
image_list = os.listdir(os.path.join(_blur_save_pth, "images"))

def split_dataset(img_name):
    _image_path = os.path.join(_blur_save_pth, "images", img_name)
    image = cv2.imread(_image_path)
    
    label_name = img_name.split(".")[0]
    label_path = os.path.join(_blur_save_pth, "labels", f"{label_name}.txt")
    label_file = open(label_path, "r", newline='')
    label_reader = csv.reader(label_file, delimiter=" ")
    
    label = []
    for l in label_reader:
        label.append(l)
    
    label_file.close()
    
    # if int(label[0]) == 0:
    if np.random.rand() < 0.7:
        # save image
        image_path = os.path.join(os.path.join(images_pth, "train"), img_name)
        cv2.imwrite(image_path, image)
        
        label_path = os.path.join(os.path.join(labels_pth, "train"), f"{label_name}.txt")
        label_file = open(label_path, "w", newline='')
        label_writer = csv.writer(label_file, delimiter=" ")
        for l in label:
            label_writer.writerow(l)
        label_file.close()
        
    elif np.random.rand() < 0.9 and np.random.rand() > 0.7:
        # save image
        image_path = os.path.join(os.path.join(images_pth, "val"), img_name)
        cv2.imwrite(image_path, image)
        
        label_path = os.path.join(os.path.join(labels_pth, "val"), f"{label_name}.txt")
        label_file = open(label_path, "w", newline='')
        label_writer = csv.writer(label_file, delimiter=" ")
        for l in label:
            label_writer.writerow(l)
        label_file.close()
        
    else:
        # save image
        image_path = os.path.join(os.path.join(images_pth, "test"), img_name)
        cv2.imwrite(image_path, image)
        
        label_path = os.path.join(os.path.join(labels_pth, "test"), f"{label_name}.txt")
        label_file = open(label_path, "w", newline='')
        label_writer = csv.writer(label_file, delimiter=" ")
        for l in label:
            label_writer.writerow(l)
        label_file.close() 
        
    label.clear()
        
    return 0

results = Parallel(n_jobs=num_processes, verbose=1)(delayed(split_dataset)(element) for element in image_list)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 260 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 1260 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Done 2660 tasks      | elapsed:    9.5s
[Parallel(n_jobs=20)]: Done 4460 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 6660 tasks      | elapsed:   23.0s
[Parallel(n_jobs=20)]: Done 9260 tasks      | elapsed:   32.0s
[Parallel(n_jobs=20)]: Done 12260 tasks      | elapsed:   42.5s
[Parallel(n_jobs=20)]: Done 15660 tasks      | elapsed:   54.1s
[Parallel(n_jobs=20)]: Done 19460 tasks      | elapsed:  1.1min
[Parallel(n_jobs=20)]: Done 23660 tasks      | elapsed:  1.4min
[Parallel(n_jobs=20)]: Done 28260 tasks      | elapsed:  1.7min
[Parallel(n_jobs=20)]: Done 33260 tasks      | elapsed:  2.0min
[Parallel(n_jobs=20)]: Done 38660 tasks      | elapsed:  2.3min
[Parallel(n_jobs=20)]: Done 44460 ta